## PPR maps with Bokeh

PPR data with GPS field added.

links

* https://towardsdatascience.com/exploring-and-visualizing-chicago-transit-data-using-pandas-and-bokeh-part-ii-intro-to-bokeh-5dca6c5ced10
* https://towardsdatascience.com/geocode-with-python-161ec1e62b89
* https://www.shanelynn.ie/the-irish-property-price-register-geocoded-to-small-areas/
* https://www.cso.ie/en/census/census2016reports/census2016boundaryfiles/
* https://python-visualization.github.io/folium/quickstart.html#Getting-Started

In [61]:
import math    
import pandas as pd
import geopandas as gpd
import json

from bokeh.io import output_file, show, output_notebook
from bokeh.models import ColumnDataSource, GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.tile_providers import get_provider, Vendors
from bokeh.plotting import figure
from bokeh.palettes import brewer

import panel as pn
import panel.widgets as pnw
pn.extension()
output_notebook()

Loading BokehJS ...

## ppr data

In [ ]:
ppr = pd.read_csv('../ppr/PPR-ALL.csv.gz')
locs = pd.read_csv('../ppr/geo_locations.csv')

In [160]:
import geopy
geopy.geocoders.options.default_user_agent = "my-application"
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
geolocator = Nominatim()
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

def get_location(x):
    address = x.address+' '+x.county    
    location = geocode(address)
    if location is None:
        return
    return pd.Series([x.address,location.latitude, location.longitude], index=['address','lat','long'])

In [162]:
locs = ppr[:20].apply(get_location,1).dropna()
locs.to_csv('../ppr/geo_locations.csv',index=False)

In [164]:
locs

,address,lat,long
0,"5 Braemor Drive, Churchtown, Co.Dublin",53.302406,-6.263796
2,"1 Meadow Avenue, Dundrum, Dublin 14",53.286736,-6.254219
4,"11 Melville Heights, Kilkenny",52.640666,-7.233922
11,"206 Philipsburgh Avenue, Marino, Dublin 3",53.371257,-6.238079
12,"22 Laverna Way, Castleknock, Dublin 15",53.373666,-6.383456
13,"23 Elton Park, Sandycove",53.284876,-6.117710
16,"29 Carysfort Woods, Blackrock",53.286926,-6.183514
19,"44 Allen Park Road, Stillorgan, County Dublin",53.285546,-6.201792


In [168]:
import folium
m = folium.Map(tiles="OpenStreetMap", location=[53.3498, -6.2603], width=600, height=400, zoom_start=11)
locs.apply(lambda x: folium.CircleMarker(location=[x["lat"], x["long"]]).add_to(m), axis=1)
m

In [60]:
def merc(Coords):
    
    lat = Coords[0]
    lon = Coords[1]
    
    r_major = 6378137.000
    x = r_major * math.radians(lon)
    scale = x/lon
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + 
        lat * (math.pi/180.0)/2.0)) * scale
    return (x, y)

In [169]:
merc((53.3498, -6.2603))

(-696893.4082131305, 7047965.391554377)

In [171]:
tile_provider = get_provider(Vendors.CARTODBPOSITRON)
# range bounds supplied in web mercator coordinates
p = figure(x_range=(-708893, -686993), y_range=(7047965, 7048985), 
           x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(tile_provider)
pn.pane.Bokeh(p)

Bokeh(Figure)

In [9]:
pn.pane.Bokeh(p)

Bokeh(Figure)

In [7]:
show(p)